# Server side `20180310a`

In [1]:
#!/usr/bin/python
import spidev
import RPi.GPIO as GPIO
import time

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


def DACCurve(Deb,Fin,CurveType)
    n = 200/5
    DACValues = []
    for k in range(n+1):
        if CurveType:
            val = int(Deb+1.0*k*(Fin-Deb)/n)
        else:
            val = int((Fin-Deb)*k**3/n**3+Deb)
        DACValues.append(val) 
    return DACValues,len(DACValues)
    
    
def CreateDACCurve(spi,DACValues):
    if len(DACValues) < 42: # to correct
        for i in range(len(DACValues)):
            if (DACValues[i] >= 0) and (DACValues[i] < 1020):
                WriteFPGA(spi,16+i,DACValues[i]/4)
            else:
                 WriteFPGA(spi,16+i,0)
    return 0

def WriteFPGA(spi,adress,value):
    spi.xfer([0xAA] )
    spi.xfer([adress] )
    spi.xfer([value] )
    
def StartUp():
    GPIO.setmode(GPIO.BCM)
    PRESET = 25
    IO4 = 26
    GPIO.setup(PRESET,GPIO.OUT)
    GPIO.setup(IO4,GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
    print "Reset 25 - Low 1s"
    GPIO.output(PRESET,GPIO.LOW)
    time.sleep(3)
    print "Reset 25 - High 0.2s"
    GPIO.output(PRESET,GPIO.HIGH)
    time.sleep(0.2)

    spi = spidev.SpiDev()
    spi.open(0,1) # CS2 - FPGA, on CE1 = IO4
    spi.mode = 0b01
    print "spi.cshigh is " + str(spi.cshigh)
    print "spi mode is " + str(spi.mode)
    spi.max_speed_hz = 2000000
    print "spi maxspeed is "+str(spi.max_speed_hz)+"hz"
    return spi

def TestSPI(spi,ncycles):
    i = 0
    while i < ncycles:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        time.sleep(0.5)
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
        time.sleep(0.5)  
        i = i+1

def LoopSPI(spi):
    while 1:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode

def LoopAcq(spi):
    while 1:
        WriteFPGA(spi,0xEB,0x00) # Doing 1 shot 
        WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
        WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software
        time.sleep(0.1) # sleep 10ms
    
def ClearMem(spi):
    WriteFPGA(spi,0xEF,0x01) # To access memory

def ConfigSPI(spi):
    # Setup FPGA values by default
    setPon(200)              # Set PulseOn
    setPulsesDelay(100)      # Set Lengh between Pon and Poff: 100ns
    setPoff(2000)            # Setting Poff 2us
    setDACConstant(20,spi)   # gain at 20mV (2%)
    setDeltaAcq(7000,spi)    # 7us
    
    WriteFPGA(spi,0xE3,0x00) # set sEEDelayACQ sEEDelayACQ MSB
    WriteFPGA(spi,0xE4,0x05) # set sEEDelayACQ EEDelayACQ LSB
    
    WriteFPGA(spi,0xE5,0x32) # set sEEPeriod MSB // durée de l'acquisition ? 
    WriteFPGA(spi,0xE6,0xC8) # set sEEPeriod LSB 0x32C8 acq (13000t * 10ns = 130us)
    WriteFPGA(spi,0xE7,0x00) # Period of one cycle MSB
    WriteFPGA(spi,0xE8,0xC3) # Period of one cycle 15 to 8
    WriteFPGA(spi,0xE9,0x50) # Period of one cycle LSB
    #WriteFPGA(spi,0xEA,0x00) # Software Trig : As to be clear by software
    WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode

    WriteFPGA(spi,0xED,0x03) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)
    WriteFPGA(spi,0xEE,0xA0) # How many cycles in countinious mode
    print "Config FPGA done!"

# Setting Poff to Acq delay sEEDelayACQ
def setDeltaAcq(DeltaAcq):
    if DeltaAcq > 255*255:
        DeltaAcq = 7000
    elif DeltaAcq < 0:
        DeltaAcq = 0
    hDA = DeltaAcq/10 
    hDAMSB, hDALSB = hDA/255 , 0x00FF&hDA 
    print "Delay between:",DeltaAcq,"ns -- ", hex(hDAMSB),hex(hDALSB)
    WriteFPGA(spi,0xE3,hDAMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE4,hDALSB) # set sEEPon LSB

def setDACConstant(mV,spi):
    if mV > 1000:
        mV = 1000
    elif mV < 0:
        mV = 0   
    hmV = mV/4
    print "Gain:", mV," mV -- ",hex(hmV)
    WriteFPGA(spi,0xEC,hmV) # Voltage gain control: 0V to 1V

def setPoff(sEEPoff,spi):
    # Sets the damping length.
    if sEEPoff > 5000:
        sEEPoff = 5000
    elif sEEPoff < 0:
        sEEPoff = 0
    POff = sEEPoff/10 
    POffMSB, POffLSB = 0xFF00&POff/0xFF,0x00FF&POff 
    print "Poff:", POff," ns -- ",hex(POffMSB),hex(POffLSB)
    WriteFPGA(spi,0xE1,POffMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE2,POffLSB) # set sEEPon LSB

def setPulsesDelay(DeltaPP,spi):
# Set Lengh between Pon and Poff
    if DeltaPP > 2500:
        DeltaPP = 2500
    elif DeltaPP < 0:
        DeltaPP = 0
    HPP =DeltaPP/10
    print  hex(HPP)
    print "Pulses delay:", DeltaPP," ns -- ",hex(HPP)
    WriteFPGA(spi,0xD0,HPP) # set sEEPon
    
def setPon(POn,spi):
    if POn > 2500:
        POn = 2500
    elif POn < 0:
        POn = 0
    HPon = POn/10
    print "Pulse width:", POn," ns -- ",hex(HPon)
    WriteFPGA(spi,0xE0,HPon) # set sEEPon

def setPeriodAcq(lEPeriod,spi):
    lEPNs = lEPeriod*1000/10 #ns
    EPNsMSB, EPNs, EPNsLSB = 0x00FF&lEPNs/(256*256),0x00FF&lEPNs/256,0x0000FF&lEPNs 
    print "Period between two acquisitions:", lEPeriod,"us --" hex(EPNsMSB),hex(EPNs),hex(EPNsLSB) 
    WriteFPGA(spi,0xE7,EPNsMSB) # Period of one cycle MSB
    WriteFPGA(spi,0xE8,EPNs) # Period of one cycle 15 to 8
    WriteFPGA(spi,0xE9,EPNsLSB) # Period of one cycle LSB
    
def SetLengthAcq(LAcq,spi):
    LAcq = 130 #us
    LAcqNs = LAcq*1000/10 #ns
    LAcqMSB, LAcqLSB = LAcqNs/256,0x00FF&LAcqNs 
    print "Acquisition length: ", LAcq, "us -- ",hex(LAcqMSB),hex(LAcqLSB)
    WriteFPGA(spi,0xE5,hDAMSB) # set sEEPon MSB
    WriteFPGA(spi,0xE6,hDALSB) # set sEEPon LSB

ImportError: No module named spidev

In [2]:
spi = StartUp()
ConfigSPI(spi)
Curve = DACCurve(100,900,True)[0] # Beginning, Ending, Linear (if False, expo)
CreateDACCurve(spi,Curve)
N = 0

Reset 25 - Low 1s
Reset 25 - High 0.2s
spi.cshigh is False
spi mode is 1
spi maxspeed is 2000000hz
Config FPGA done!


In [55]:
#LoopSPI(spi)
#LoopAcq(spi)

In [3]:
TestSPI(spi,3) # LED2 clignote 3x

In [69]:
# Setup the different variables
setPon(200)              # Set PulseOn
setPulsesDelay(100)      # Set Lengh between Pon and Poff: 100ns
setPoff(2000)            # Setting Poff 2us
setDACConstant(20,spi)   # gain at 20mV (2%)
setDeltaAcq(7000,spi)    # 7us
setPeriodAcq(1000,spi)   # 1ms between two acquisitions
SetLengthAcq(130,spi)    # setting the length of one acquisition (in us) 
# Numbers of repeats
NAcq = 10                # Doing 10 repeats
WriteFPGA(spi,0xEE,NAcq) # Number of acquisitions

0x14
0xa
Poff: 0x0 0xc8
hDA: 0x2 0xbc
Acq: 0x32 0xc8
Period: 0x1 0x86 0xa0


In [7]:
f = 0x01
LAcq = 130 #us
WriteFPGA(spi,0xED,f) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)
VGA = 0x44
NAcTrig = 200 # 200 acquisitions
WriteFPGA(spi,0xEC,VGA)
WriteFPGA(spi,0xEB,0x01) # Doing several lines
WriteFPGA(spi,0xEE,NAcTrig) # Acquire 100 lines
WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software

time.sleep(0.2) # sleep 10ms

Fech = int(64/((1+f)))
print Fech
Nacq = LAcq * Fech * NAcTrig
print "-> "+str(Nacq) + ' samples' 

10
-> 260000 samples


In [10]:
LAcq * Fech

1300

In [8]:
spi.max_speed_hz = 9000000
A = []
limit = 2*Nacq-10
x = 20000
print limit
print limit/x
for i in range(limit):
    A.append ( spi.xfer([0x00] )[0] )  
    if not(i%x):
        print len(A)/x
a = np.asarray(A).astype(int)
print 'acq done'
np.savetxt( "FI-"+str(NAcTrig)+"-"+str(N)+"-VGA@"+str(hex(VGA))+"-spimode"+str(spi.mode)+"-"+str(Fech)+"msps.csv", a.astype(int), fmt='%i', delimiter=";")
N = N+1

519990
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
acq done


In [9]:
519990/2

259995

In [25]:
spi.close() 

In [81]:
f = 0x02
WriteFPGA(spi,0xED,f) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)
VGA = 0x22
WriteFPGA(spi,0xEC,VGA)
WriteFPGA(spi,0xEB,0x00) # Doing several lines 
WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software

time.sleep(0.2) # sleep 10ms

Fech = int(64/((1+f)))
print Fech
Nacq = LAcq * Fech
print "-> "+str(Nacq) + ' samples' 

21
-> 2730 samples


In [82]:
A = []
for i in range(2*Nacq-10):
    A.append ( spi.xfer([0x00] )[0] )  
a = np.asarray(A).astype(int)

np.savetxt( "One-"+str(N)+"-VGA@"+str(hex(VGA))+"-spimode"+str(spi.mode)+"-"+str(Fech)+"msps.csv", a, delimiter=";")
N = N+1

In [110]:
LAcq * 64.0/3

2773.3333333333335